# Desafio Técnico Itaú/CNpq - Previsão de Custos de VMs na Azure (Versão Refatorada)
*Autora: Ana Leticia de Araujo*

Este notebook implementa um pipeline de dados e Machine Learning para prever os custos de Máquinas Virtuais da Azure. O código foi refatorado seguindo princípios de Clean Code para garantir modularidade, legibilidade e fácil manutenção, ao mesmo tempo em que preserva a estrutura narrativa e explicativa de um notebook.

**A estrutura do notebook é a seguinte:**
1.  **Configuração Centralizada:** Todos os parâmetros e constantes do projeto são definidos em um único lugar.
2.  **Definição das Funções do Pipeline:** Todas as funções que executam as tarefas (coleta, limpeza, treinamento, etc.) são definidas em uma única "célula de ferramentas".
3.  **Execução do Pipeline Passo a Passo:** As funções são chamadas em sequência, célula a célula, para executar o pipeline, permitindo a inspeção dos resultados em cada etapa.

In [ ]:
# --- ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS ---
import requests
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import List, Dict, Tuple, Optional

## 1. Configuração Centralizada do Pipeline
Nesta célula, todas as configurações, como URLs, nomes de arquivos e parâmetros do modelo, são definidas para fácil manutenção.

In [ ]:
# Centraliza todas as configurações e "strings mágicas" em um único lugar
CONFIG = {
    "api_url": "https://prices.azure.com/api/retail/prices?$filter=serviceName eq 'Virtual Machines' and (armRegionName eq 'brazilsoutheast' or armRegionName eq 'eastus') and priceType eq 'Consumption'",
    "columns_to_select": ['armSkuName', 'retailPrice', 'armRegionName', 'vcpu', 'memoria_gb'],
    "column_rename_map": {
        'armSkuName': 'tipo_instancia',
        'retailPrice': 'preco_hora',
        'armRegionName': 'regiao'
    },
    "region_translation_map": {
        'eastus': 'Leste dos EUA',
        'brazilsoutheast': 'Sudeste do Brasil'
    },
    "exploratory_plot_path": "analise_exploratoria_vcpu_por_regiao.png",
    "dashboard_plot_path": "dashboard_performance_modelo.png",
    "results_csv_path": "dados_tratados_e_resultados.csv",
    "random_state": 42,
    "test_size": 0.2
}

## 2. Definição das Funções do Pipeline
Aqui definimos todas as funções que servirão como os blocos de construção do nosso pipeline. Cada função tem uma única responsabilidade.

### 2.1 Funções de Coleta e Processamento de Dados
Iniciamos com as funções que formam o núcleo do nosso pipeline de ETL (Extração, Transformação e Carga). A primeira é responsável pela **extração** dos dados brutos da API.

In [ ]:
def fetch_data_from_api(api_url: str) -> Optional[pd.DataFrame]:
    """Busca e pagina os dados da API da Azure, retornando um DataFrame."""
    print("--- Coletando dados da API... ---")
    all_items = []
    page_number = 1
    
    while api_url:
        print(f"Buscando dados da página {page_number}...")
        try:
            response = requests.get(api_url)
            response.raise_for_status()
            data = response.json()
            all_items.extend(data.get('Items', []))
            api_url = data.get('NextPageLink')
            page_number += 1
        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar a API: {e}")
            return None
            
    if not all_items:
        print("Nenhum dado foi retornado pela API.")
        return None
            
    print(f"SUCESSO: {len(all_items)} registros de preços coletados.")
    return pd.DataFrame(all_items)

---
A seguir, definimos uma função auxiliar (`helper function`) crucial para a etapa de **transformação**. Esta função atuará como um "decodificador" para extrair a contagem de vCPUs e a quantidade de memória a partir do nome técnico da instância (`armSkuName`).

In [ ]:
def get_specs_from_name(sku_name: str) -> Tuple[Optional[int], Optional[float]]:
    """Extrai vCPU e Memória do nome da instância (armSkuName)."""
    memory_ratio_map = {'B': 2, 'D': 4, 'E': 8, 'F': 2, 'M': 8, 'DC': 4}
    match = re.search(r'Standard_([A-Z]+)(\d+)', sku_name)
    if not match: return None, None
    family, vcpu = match.group(1), int(match.group(2))
    ratio = memory_ratio_map.get(family, 4)
    memory = float(vcpu * ratio)
    return vcpu, memory


---
Com a função auxiliar pronta, construímos a principal função de limpeza e engenharia de features. Ela orquestrará todo o processo de **transformação**: aplicará filtros, usará a `get_specs_from_name` para criar novas colunas e preparará o DataFrame final.

In [ ]:
def clean_and_feature_engineer(df: pd.DataFrame) -> Optional[pd.DataFrame]:
    """Aplica a limpeza, engenharia de features e transformações no DataFrame."""
    print("\n--- Limpando dados e aplicando engenharia de features... ---")
    query_filter = (
        (df['unitOfMeasure'] == '1 Hour') &
        (df['armSkuName'].str.contains('Standard_D|Standard_B|Standard_E', na=False)) &
        (~df['meterName'].str.contains('Spot', na=False)) &
        (~df['meterName'].str.contains('Low Priority', na=False)) &
        (~df['productName'].str.contains('Promo', na=False))
    )
    df_clean = df[query_filter].copy()

    specs_df = df_clean['armSkuName'].apply(get_specs_from_name).apply(pd.Series)
    df_clean['vcpu'], df_clean['memoria_gb'] = specs_df[0], specs_df[1]

    df_final = df_clean[CONFIG["columns_to_select"]].rename(columns=CONFIG["column_rename_map"]).dropna().copy()
    
    if df_final.empty: return None
        
    df_final['regiao'] = df_final['regiao'].map(CONFIG["region_translation_map"])
    return df_final

### 2.2 Funções de Visualização, Modelagem e Resultados
Agora definimos as funções para as etapas restantes: Análise Exploratória, Modelagem, Avaliação e Salvamento.

In [ ]:
def generate_exploratory_plot(df: pd.DataFrame):
    """Gera e salva o gráfico de análise exploratória."""
    print("\n--- Gerando análise exploratória visual... ---")
    plt.style.use('seaborn-v0_8-whitegrid')
    g = sns.lmplot(data=df, x='vcpu', y='preco_hora', hue='regiao', x_estimator=np.mean, height=7, aspect=1.5, palette='viridis')
    g.fig.suptitle('Tendência de Custo Médio por Quantidade de vCPUs', fontsize=18, y=1.03, weight='bold')
    g.set_axis_labels('Quantidade de vCPUs', 'Custo Médio por Hora (USD)', fontsize=14)
    g.legend.set_title("Região")
    plt.savefig(CONFIG["exploratory_plot_path"], dpi=300)
    print(f"Gráfico exploratório salvo em: {CONFIG['exploratory_plot_path']}")
    plt.show()

---
A função seguinte encapsula toda a lógica de **treinamento do modelo** de regressão.

In [ ]:
def train_model(df: pd.DataFrame) -> Tuple[object, pd.DataFrame, pd.Series, np.ndarray]:
    """Prepara os dados, treina o modelo e retorna os resultados."""
    print("\n--- Preparando dados e treinando o modelo... ---")
    df_processed = pd.get_dummies(df, columns=['regiao'], prefix='reg')
    df_processed.drop('tipo_instancia', axis=1, inplace=True)
    
    X = df_processed.drop('preco_hora', axis=1)
    y = df_processed['preco_hora']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=CONFIG["test_size"], random_state=CONFIG["random_state"])
    
    model = RandomForestRegressor(n_estimators=100, random_state=CONFIG["random_state"])
    model.fit(X_train, y_train)
    print("Modelo treinado com sucesso!")
    predictions = model.predict(X_test)
    
    return model, X_test, y_test, predictions

---
Esta função é responsável por **avaliar a performance** do modelo treinado e gerar o dashboard visual com os resultados.

In [ ]:
def evaluate_and_plot(y_test: pd.Series, predictions: np.ndarray, df_final: pd.DataFrame, X_test: pd.DataFrame):
    """Calcula métricas, gera e salva o dashboard de performance."""
    print("\n--- Avaliando performance e gerando dashboard... ---")
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print(f"Erro Quadrático Médio (MSE): {mse:.4f}")
    print(f"Coeficiente de Determinação (R²): {r2:.4f}")

    plot_data = X_test.copy()
    plot_data.loc[:, 'preco_real'] = y_test
    plot_data.loc[:, 'preco_previsto'] = predictions
    plot_data.loc[:, 'residuos'] = y_test - predictions
    plot_data.loc[:, 'regiao'] = df_final.loc[X_test.index, 'regiao']

    plt.style.use('seaborn-v0_8-talk')
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 14))
    fig.suptitle('Dashboard de Performance do Modelo de Regressão', fontsize=22, weight='bold', y=1.02)
    
    sns.scatterplot(data=plot_data, x='preco_real', y='preco_previsto', hue='regiao', palette='viridis', alpha=0.8, s=100, ax=ax1)
    ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red', linewidth=2, label='Previsão de custo ideal')
    ax1.set_title('Gráfico de Dispersão: Preço Real vs. Previsão', fontsize=16)
    ax1.legend(title='Região')
    ax1.text(0.05, 0.95, f'$R^2 = {r2:.4f}$\nMSE = {mse:.5f}', transform=ax1.transAxes, fontsize=14, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', fc='lightgray', alpha=0.6))

    sns.scatterplot(data=plot_data, x='preco_previsto', y='residuos', hue='regiao', palette='viridis', alpha=0.7, s=100, legend=False, ax=ax2)
    ax2.axhline(y=0, color='r', linestyle='--')
    ax2.set_title('Gráfico de Resíduos (Análise de Erros)', fontsize=16)

    plt.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.savefig(CONFIG["dashboard_plot_path"], dpi=300)
    print(f"Dashboard salvo em: {CONFIG['dashboard_plot_path']}")
    plt.show()

---
Finalmente, uma função para **salvar os resultados** numéricos em um arquivo `.csv`.

In [ ]:
def save_results(X_test: pd.DataFrame, y_test: pd.Series, predictions: np.ndarray, df_final: pd.DataFrame):
    """Salva os resultados finais em um arquivo CSV."""
    print("\n--- Salvando resultados em CSV... ---")
    df_resultados = X_test.copy()
    df_resultados['preco_real'] = y_test
    df_resultados['preco_previsto'] = predictions
    df_resultados['regiao'] = df_final.loc[X_test.index, 'regiao']
    df_resultados.to_csv(CONFIG["results_csv_path"], index=False)
    print(f"Arquivo de resultados salvo em: {CONFIG['results_csv_path']}")

## 3. Execução do Pipeline Passo a Passo
As células a seguir orquestram a execução do pipeline, chamando as funções definidas acima e permitindo a visualização dos resultados de cada etapa principal.

### ETAPA 1: Coleta de Dados

In [ ]:
# Chama a função para buscar os dados da API
df_raw = fetch_data_from_api(CONFIG["api_url"])

# Exibe as primeiras linhas do DataFrame bruto, se a coleta for bem-sucedida
if df_raw is not None:
    display(df_raw.head())

### ETAPA 2: Limpeza e Engenharia de Features

In [ ]:
# Chama a função para limpar os dados e criar novas features
if df_raw is not None:
    df_final = clean_and_feature_engineer(df_raw)
    if df_final is not None:
        display(df_final.head())

### ETAPA 2.5: Análise Exploratória Visual

In [ ]:
# Gera o gráfico para explorar a relação entre vCPU e preço
if df_final is not None:
    generate_exploratory_plot(df_final)

### ETAPAS 3 e 4: Treinamento do Modelo

In [ ]:
# Treina o modelo e obtém as previsões e dados de teste
if df_final is not None:
    model, X_test, y_test, predictions = train_model(df_final)

### ETAPA 5: Avaliação e Visualização dos Resultados

In [ ]:
# Gera o dashboard de performance com os resultados do modelo
if 'model' in locals():
    evaluate_and_plot(y_test, predictions, df_final, X_test)

### ETAPA 6: Geração do Arquivo de Resultados

In [ ]:
# Salva o CSV final com os resultados
if 'model' in locals():
    save_results(X_test, y_test, predictions, df_final)
    print("\n--- DESAFIO CONCLUÍDO ---")